<a href="https://colab.research.google.com/github/obeabi/SdsNovember2020Challenge/blob/main/Main_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Written by Abiola Obembe
# SDS October Challenge #3 - Fraudulent Job Adverts
## 2021-01-19

## Data Preprocessing

In [2]:
# Import category_encoders package
!pip install category_encoders

     |████████████████████████████████| 81kB 4.3MB/s 


In [13]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import  zscore
import category_encoders as ce
import warnings
warnings.filterwarnings("ignore", category= FutureWarning)

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

print('Libraries installed successfully!')

np.random.seed(0)

Libraries installed successfully!


In [14]:
# Importing the training dataset

df_train = pd.read_csv('public_jobs.csv', engine= 'python', encoding= 'latin1')

df_train.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,Sales Professional - Buiding Products,"CA, BC, Langley",Langley,60000-90000,Great Customer Service from Start to FinishWha...,"Centra Windows an established, employee-owned ...",NaN,NaN,0,1,1,Full-time,Mid-Senior level,NaN,Building Materials,Sales,0
1,Senior Software Engineer,"GB, , London",Engineering,NaN,We believe cities are complicated. And your mo...,"Londoners, New Yorkers, Parisians, and Berline...",We're looking for someone with several years' ...,NaN,0,1,1,Full-time,NaN,NaN,Computer Software,Engineering,0
2,Director of Operations (SCM & Manufacturing),"US, WA, Bellingham",Operations,50000-70000,POS-X is a rapidly growing point-of-sale hardw...,POS-X is a rapidly growing point-of-sale hardw...,Extensive Mathematical and/or Financial Backgr...,"$50 - $70k DOE and Performance + Medical, 401k...",0,1,1,Full-time,Director,Bachelor's Degree,Computer Hardware,Supply Chain,0
3,Data Center Migration App Lead for FULL-TIME O...,"US, PA, Harrisburg",NaN,0-0,NaN,Data Center Migration Application Lead / Archi...,NaN,NaN,0,0,0,Full-time,Director,Bachelor's Degree,Information Technology and Services,Information Technology,1
4,Data Entry Assistant - Intern,"HK, , Hong Kong",Marketing,8000-14000,"At Hayes-Corp, we create the fun stuff.Â With...","Hayes Corp is looking for a patient, meticulou...",Degree in any discipline.Familiar with Microso...,"Opportunity to grow with the company, as data ...",0,1,0,Other,Entry level,Bachelor's Degree,Internet,Marketing,0


In [15]:
# priniting the details about the train dataset

print("\nEDA on Training Set\n")
print("#"*30)

print("\nFeatures/Columns : \n", df_train.columns)
print("\n\nNumber of Features/Columns : ", len(df_train.columns))
print("\nNumber of Rows : ",len(df_train))
print("\n\nData Types :\n", df_train.dtypes)

print("\n Contains NaN/Empty cells : ", df_train.isnull().values.any())

print("\n Total empty cells by column :\n", df_train.isnull().sum(), "\n\n")

print("\n Perecentage of missing values in train datframe is: \n", str(100*(df_train.isnull().sum().sum()/(df_train.shape[0]* df_train.shape[1])))+'%')


EDA on Training Set

##############################

Features/Columns : 
 Index(['title', 'location', 'department', 'salary_range', 'company_profile',
       'description', 'requirements', 'benefits', 'telecommuting',
       'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')


Number of Features/Columns :  17

Number of Rows :  14304


Data Types :
 title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
dtype: objec

In [16]:
# Importing the test dataset

df_test = pd.read_csv('pred_jobs.csv', engine= 'python', encoding= 'latin1')

df_test.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function
0,Part-time Pizza Delivery Drivers - Wallasey,"GB, WRL, Wallasey",,NaN,Take Out Brands is a food franchise business w...,Papa Johnâs is one of the worldâs biggest ...,You must be 19 years old or above â and will...,Pay rates will be Â£6.31 / hour (21 and over)Â...,0,1,1,Part-time,Not Applicable,NaN,Retail,NaN
1,Director of Contact Center Operations,"US, AZ, Tempe",Operations,NaN,"J-Curve Technologies,Â a privately held Arizon...",J-Curve Technologies is currently in search of...,Experience and Education: Minimum of 5-8 years...,Full time employees enjoy a comprehensive bene...,0,1,1,Full-time,Not Applicable,NaN,NaN,Management
2,Technical Solution Consultant,"US, IL, Chicago",NaN,80000-105000,Retailers around the world are constantly work...,Job Title: Technical Solution ConsultantDepart...,NaN,NaN,0,0,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology
3,Vacancies At The Cafe Royal Hotel London,"GB, EAW, london",Hospitality,NaN,NaN,"AVAILABLE POSITIONS:Catering staffs, managers,...",Word Perfect;Typing;Public Speaking;Scheduling...,The Hotel Management would be responsible to c...,0,0,0,NaN,NaN,NaN,NaN,NaN
4,Car Dealer Sales Manager/Customer Service,"GR, I, Athens",NaN,NaN,Ilias Sotiropoulos first went into business as...,Full time Sales Manager/Customer Service for a...,Max 35 years old.Living close to the workplace...,NaN,0,1,0,Full-time,Mid-Senior level,Unspecified,Retail,Sales


In [8]:

# priniting the details about the test dataset

print("#"*30)
print("\nEDA on Test Set\n")
print("#"*30)


print("\nFeatures/Columns : \n",df_test.columns)
print("\n\nNumber of Features/Columns : ",len(df_test.columns))
print("\nNumber of Rows : ",len(df_test))
print("\n\nData Types :\n", df_test.dtypes)
print("\n Contains NaN/Empty cells : ", df_test.isnull().values.any())
print("\n Total empty cells by column :\n", df_test.isnull().sum())
print("\n Perecentage of missing values in test datframe is: \n", str(100*(df_test.isnull().sum().sum()/(df_test.shape[0]* df_test.shape[1])))+'%')

##############################

EDA on Test Set

##############################

Features/Columns : 
 Index(['title', 'location', 'department', 'salary_range', 'company_profile',
       'description', 'requirements', 'benefits', 'telecommuting',
       'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function'],
      dtype='object')


Number of Features/Columns :  16

Number of Rows :  3576


Data Types :
 title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
dtype: object

 Contains NaN/Empty ce

In [9]:
# Retrrieve information from train dataframe
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14304 entries, 0 to 14303
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                14304 non-null  object
 1   location             14029 non-null  object
 2   department           5122 non-null   object
 3   salary_range         2325 non-null   object
 4   company_profile      11651 non-null  object
 5   description          14303 non-null  object
 6   requirements         12155 non-null  object
 7   benefits             8546 non-null   object
 8   telecommuting        14304 non-null  int64 
 9   has_company_logo     14304 non-null  int64 
 10  has_questions        14304 non-null  int64 
 11  employment_type      11542 non-null  object
 12  required_experience  8674 non-null   object
 13  required_education   7837 non-null   object
 14  industry             10404 non-null  object
 15  function             9144 non-null   object
 16  frau

In [10]:
# Retrieve statistics of train dataframe
df_train.describe()

,telecommuting,has_company_logo,has_questions,fraudulent
count,14304.000000,14304.000000,14304.000000,14304.000000
mean,0.043554,0.794393,0.492799,0.049567
std,0.204108,0.404158,0.499966,0.217055
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


## EDA